In [ ]:
import sys
sys.path.append("..")
import mdtraj as md
import os
import numpy as np
from Accuracy_metrics_pipeline import *
from analysis_plots import *


In [3]:
base_gen_dir = "../not_shuffle_not_allframes"
base_ref_dir = "../../data/palantir_data"
pdb = f"{base_ref_dir}/backbone.pdb"
out_base = "../Analysis_results"


In [4]:
def get_paths(fraction, replica, protein):
    frac_tag = int(fraction * 100)

    gen_xtc = (
        f"{base_gen_dir}/{protein}/{protein}_rep_{replica}/"
        f"fraction_{frac_tag}/generated_frac_{frac_tag}.xtc"
    )

    ref_xtc = (
        f"{base_ref_dir}/{protein}/{protein}_rep_{replica}_backbone.xtc"
    )

    pdb = f"{base_ref_dir}/{protein}/backbone.pdb"

    out_dir = (
        f"{out_base}/{protein}/rep_{replica}/fraction_{frac_tag}"
    )

    return gen_xtc, ref_xtc, pdb, out_dir



In [ ]:
def run_all_analyses(gen_xtc, ref_xtc, pdb, out_dir):

    gen_traj, ref_traj = data_preprocessing(gen_xtc, ref_xtc, pdb)

    rmsd_gen, rmsd_ref = rmsd_internal(gen_traj, ref_traj)
    res_ids, rmsf_ref, rmsf_gen = rmsf_md_vs_gen(gen_traj, ref_traj)
    rg_gen, rg_ref = radius_of_gyration(gen_traj, ref_traj)
    phi_r, psi_r, phi_g, psi_g = compute_phi_psi(gen_traj, ref_traj)
    cm_ref = compute_contact_map(ref_traj)
    cm_gen = compute_contact_map(gen_traj)
    X_ref_pca, X_gen_pca = pca(gen_traj, ref_traj)

    save_figure(plot_rmsd(rmsd_ref, rmsd_gen), f"{out_dir}/rmsd.png")
    save_figure(plot_rmsf(res_ids, rmsf_ref, rmsf_gen), f"{out_dir}/rmsf.png")
    save_figure(plot_rg(rg_ref, rg_gen), f"{out_dir}/rg.png")
    save_figure(plot_ramachandran(phi_r, psi_r, phi_g, psi_g), f"{out_dir}/rama.png")
    save_figure(plot_contact_map(cm_ref, "Contact Map MD"), f"{out_dir}/cm_md.png")
    save_figure(plot_contact_map(cm_gen, "Contact Map Gen"), f"{out_dir}/cm_gen.png")
    save_figure(plot_pca(X_ref_pca, X_gen_pca), f"{out_dir}/pca.png")


In [6]:
fractions = [0.3, 0.4, 0.5]
replicas = [0, 1, 2]
proteins = ["3UTQ","5N1Y","5C0F","pep_free"]

for prot in proteins:
    for fraction in fractions:
        for replica in replicas:

            gen_xtc, ref_xtc, pdb, out_dir = get_paths(
                fraction, replica, prot
            )
            print(
                f"[RUN] protein={prot} | rep={replica} | fraction={fraction}"
            )

            run_all_analyses(gen_xtc, ref_xtc, pdb, out_dir)



[RUN] protein=3UTQ | rep=0 | fraction=0.3
[RUN] protein=3UTQ | rep=1 | fraction=0.3
[RUN] protein=3UTQ | rep=2 | fraction=0.3
[RUN] protein=3UTQ | rep=0 | fraction=0.4
[RUN] protein=3UTQ | rep=1 | fraction=0.4
[RUN] protein=3UTQ | rep=2 | fraction=0.4
[RUN] protein=3UTQ | rep=0 | fraction=0.5
[RUN] protein=3UTQ | rep=1 | fraction=0.5
[RUN] protein=3UTQ | rep=2 | fraction=0.5
[RUN] protein=5N1Y | rep=0 | fraction=0.3
[RUN] protein=5N1Y | rep=1 | fraction=0.3
[RUN] protein=5N1Y | rep=2 | fraction=0.3
[RUN] protein=5N1Y | rep=0 | fraction=0.4
[RUN] protein=5N1Y | rep=1 | fraction=0.4
[RUN] protein=5N1Y | rep=2 | fraction=0.4
[RUN] protein=5N1Y | rep=0 | fraction=0.5
[RUN] protein=5N1Y | rep=1 | fraction=0.5
[RUN] protein=5N1Y | rep=2 | fraction=0.5
[RUN] protein=5C0F | rep=0 | fraction=0.3
[RUN] protein=5C0F | rep=1 | fraction=0.3
[RUN] protein=5C0F | rep=2 | fraction=0.3
[RUN] protein=5C0F | rep=0 | fraction=0.4
[RUN] protein=5C0F | rep=1 | fraction=0.4
[RUN] protein=5C0F | rep=2 | fract